# Artificial and Computational Intelligence (Assignment - 2)

## Problem Statement

As part of the 2nd Assignment, we'll try and predict the Part of Speech (POS) tag for each word in a provided sentence.

You are required to build a model using Hidden Markov Models which would help you predict the POS tags for all words in an utterance.

### What is a POS tag?

In corpus linguistics, part-of-speech tagging (POS tagging or PoS tagging or POST), also called grammatical tagging or word-category disambiguation, is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech, based on both its definition and its context—i.e., its relationship with adjacent and related words in a phrase, sentence, or paragraph. A simplified form of this is commonly taught to school-age children, in the identification of words as nouns, verbs, adjectives, adverbs, etc.

### Dataset

The dataset can be downloaded from https://drive.google.com/open?id=1345iaxqTImJN6mKGh_c1T5n2OWumpyYz. You can access it only using your BITS IDs.

#### Dataset Description
##### Sample Tuple
b100-5507

Mr.	NOUN
<br>
Podger	NOUN
<br>
had	VERB
<br>
thanked	VERB
<br>
him	PRON
<br>
gravely	ADV
<br>
,	.
<br>
and	CONJ
<br>
now	ADV
<br>
he	PRON
<br>
made	VERB
<br>
use	NOUN
<br>
of	ADP
<br>
the	DET
<br>
advice	NOUN
<br>
.	.
<br>
##### Explanation
The first token "b100-5507" is just a key and acts like an identifier to indicate the beginning of a sentence.
<br>
The other tokens have a (Word, POS Tag) pairing.

__List of POS Tags are:__
.
<br>
ADJ
<br>
ADP
<br>
ADV
<br>
CONJ
<br>
DET
<br>
NOUN
<br>
NUM
<br>
PRON
<br>
PRT
<br>
VERB
<br>
X

__Note__
<br>
__.__ is used to indicate special characters such as '.', ','
<br>
__X__ is used to indicate vocab not part of Enlish Language mostly.
Others are Standard POS tags.

### Evaluation
We wish to evaluate based on 
- coding practices being followed
- commenting to explain the code and logic behind doing something
- your understanding and explanation of data
- how good the model would perform on unseen data.

### Train-Test Split
Let us use a 80-20 split of our data for training and evaluation purpose.



In [1]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict,OrderedDict,namedtuple
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import random

In [2]:
#Read data
Sentence = namedtuple("Sentence", "words tags")


def read_data_infile(filename):
    """Read tagged sentence data"""
    with open(filename, 'r') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

In [3]:
data_dict=read_data_infile('data.txt')

In [4]:
data_dict['b100-935']

Sentence(words=('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.'), tags=('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.'))

In [5]:
tags=['ADJ' ,'ADP' ,'ADV' ,'CONJ' ,'DET' ,'NOUN' ,'NUM' ,'PRON' ,'PRT' ,'VERB' ]

In [6]:
#Pre-process data (Whatever you feel might be required)
def read_required_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

In [7]:
all_tags=read_required_tags('data.txt')

In [8]:
#Data Description
keys = tuple(data_dict.keys())

In [9]:
# vocabulary of all unique words 
wordset = frozenset(chain(*[s.words for s in data_dict.values()]))

In [10]:
## word and tags 
sequences_word = tuple([data_dict[key].words for key in keys])
sequences_tag = tuple([data_dict[key].tags for key in keys])

In [11]:
import collections
dt = collections.defaultdict(list)
def Subset(sentences,key):
    return {x: sentences[x] for x in key}

In [12]:
## train test split 

key_list = list(keys)
random.shuffle(key_list)
split = int( 0.8 * len(key_list))
training_data = Subset(data_dict, key_list[:split])
test_data = Subset(data_dict, key_list[split:])
print(len(training_data),len(test_data))

45872 11468


In [14]:
print("Training set has {} sentences.".format(len([data_dict[x] for x in training_data.keys()])))
print("Test set has {} sentences.\n".format(len([data_dict[x] for x in  test_data.keys()])))


Training set has 45872 sentences.
Test set has 11468 sentences.



In [15]:
def get_allSets(my_dict,key_list):
    word_sequences = tuple([my_dict[key].words for key in key_list])
    tag_sequences = tuple([my_dict[key].tags for key in key_list])
    wordset = frozenset(chain(*word_sequences))
    tagset = frozenset(chain(*tag_sequences))
    return (word_sequences,tag_sequences,wordset,tagset)

In [16]:
train_instances = get_allSets(training_data,training_data.keys())


In [17]:
################ 2. COUNT NUMBER OF EACH TAG IN ENTIRE CORPUS #################
########################  single_tag_counts[tag] = k  #########################
all_sentences = [list(sentence) for sentence in [[data_dict[x] for x in training_data.keys()]]]
all_tags = chain.from_iterable(all_sentences)
single_tag_counts = dict(Counter(all_tags))

In [18]:
############## 3. COUNT NUMBER OF EACH TAG PAIR IN ENTIRE CORPUS ##############
####################  pair_tag_counts[(tag_1, tag_2)] = k  ####################
sentences = [s for s in all_sentences if len(s) > 1]  # discard any sequences of length 1
pairs = []
for s in sentences:
    pairs.extend([(s[i-1], s[i]) for i in range(1, len(s))])

pair_tag_counts = dict(Counter(pairs))

if len(pair_tag_counts) < len(train_instances[3])**2:
    for tag1 in train_instances[3]:
        for tag2 in train_instances[3]:
            if (tag1, tag2) not in pair_tag_counts:
                pair_tag_counts[(tag1, tag2)] = 0

In [19]:
## 4. COUNT NUMBER OF EACH TAG APPEARING IN THE BEGINNING or END OF SENTENCE ##
#########################  start_tag_counts[tag] = k  #########################
##########################  end_tag_counts[tag] = k  ##########################
start_tag_counts = dict(Counter([sentence[0] for sentence in [[data_dict[x] for x in training_data.keys()]] ]))
end_tag_counts = dict(Counter([sentence[-1] for sentence in [[data_dict[x] for x in training_data.keys()]]]))

### if any tag has NO sentences starting/ending with it, set its value to 0:
if len(start_tag_counts) < len(train_instances[3]):
    for tag in train_instances[3]:
        if tag not in start_tag_counts:
            start_tag_counts[tag] = 0

if len(end_tag_counts) < len(train_instances[3]):
    for tag in train_instances[3]:
        if tag not in end_tag_counts:
            end_tag_counts[tag] = 0

In [21]:
training_data['b100-5507']

Sentence(words=('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.'), tags=('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.'))

In [35]:
train_tag_sequences=train_instances[1]
train_word_sequences=train_instances[0]

In [31]:
################## 5. COUNT NUMBER OF (TAG_i, WORD_i) PAIRS ###################
#######################  pair_counts[tag][word] = k  ##########################
pair_counts = defaultdict(lambda: defaultdict(lambda : 0))

train_tag_sequences=train_instances[1]
for sentence_idx, sentence in enumerate([training_data[x] for x in training_data.keys()]):
    for word_idx, tag in enumerate(sentence):
        word = train_tag_sequences[word_idx]
        pair_counts[tag][word] += 1

In [34]:
#HMM Model Goes Here
HMM_model = HiddenMarkovModel(name = "HMM-Tagger")
tag_states = [] 
for tag in train_instances[3]:
    tag_emissions = DiscreteDistribution({word:pair_counts[tag][word]/single_tag_counts[tag] \
                                          for word in train_instances[2]})
    tag_state = State(tag_emissions, name = tag)
    tag_states.append(tag_state)
    HMM_model.add_states(tag_state)

In [33]:
#Model Accuracy Evaluation
n_sentences = len(data.training_set.keys)

for tag_state1 in tag_states:
    for tag_state2 in tag_states:
        tag1, tag2 = tag_state1.name, tag_state2.name
        HMM_model.add_transition(HMM_model.start, tag_state1, start_tag_counts[tag1]/n_sentences)
        HMM_model.add_transition(tag_state1, HMM_model.end, end_tag_counts[tag1]/single_tag_counts[tag1])
        HMM_model.add_transition(tag_state1, tag_state2, pair_tag_counts[(tag1,tag2)]/single_tag_counts[tag1])
    
HMM_model.bake()    

In [37]:
#Adds code blocks wherever you feel necessary

####################### 8. MAKE PREDICTIONS ON TEST SET #######################
test_correct = 0 
test_count = 0   

### ITERATE PER SENTENCE
for words, true_tags in zip(train_tag_sequences, train_word_sequences):
    try: 
        # Only consider words contained in training set's vocab
        _, viterbi_path = HMM_model.viterbi([w if w in train_instances[2] else 'nan' for w in words])
        predicted_tags = [state[1].name for state in viterbi_path[1:-1]] 
        test_correct += sum(pred == true for pred, true in zip(predicted_tags, true_tags))
        
        if print_i == 101: # print a sample result
            print("Test Sentence: \n", words)
            print()
            print("Predicted Tags: \n", predicted_tags)
            print()
            print("True Tags: \n", true_tags)
            print_i += 1
    except:
        pass
    test_count += len(words)

test_acc = test_correct/test_count
print("\nTest Accuracy: {:.2f}%".format(100 * test_acc))


Test Accuracy: 78.38%


### Happy Coding!